In [1]:
#process a object that contains information about a person
def process_people_group(people_group_els, this_ref_dict):
    
    #scroll through the people objects
    for pg_els in people_group_els:
        
        #define the person dictionary
        person_dict = {}
        
        #run through the elements in this person object
        for span_el in pg_els:
            
            #print(span_el)
            
            #check if we have reached the et al.
            if span_el.name == 'i':
                person_dict['etal'] = 'etal'
                
                continue
            
            #store the person's details
            person_dict[span_el['class'][0]] = span_el.text

        #store this person
        this_ref_dict['people'].append(person_dict)
        
    return this_ref_dict

#process an list element which is a reference
def process_reference_element(l):
    
    #set up the empty reference dictionary to be safe
    ref_dict = {}
    
    #run through the elements of this reference element
    for t in l.children:

        #check if the item is whatever a navigable string is
        if 'bs4.element.NavigableString' in str(t.__class__):
            #move on if it does
            continue

        #if we have the label element grab it and form the background of our element
        if t.name == 'a':
            ref_dict = {'people':[],
                       'label':t['name'],
                       'ref_no':int(t['name'][1:])}
            continue

        #go through the children of the reference to grab the reference's elements
        for u in t.children:
            
            #check that the element we have is not a NavigableString
            if 'bs4.element.NavigableString' in str(u.__class__):
                #move on if it does
                continue

            #if we have a person group then we want to use our extraction tool
            if 'person-group' in u['class'][0]:

                #process it as a people element
                ref_dict = process_people_group(u, ref_dict)

            else: 
                
                #find to see if there are links in here
                As = u.findAll('a')
                
                #if there are links we process it as a link object
                if As != []:

                    #if we have a reference link we store the text as well
                    #as the link
                    ref_dict[u['class'][0]] = {'link':As[0]['href'],
                                               'text':u.text}

                else:
                    #we have a plain piece of information, lets just store it
                    ref_dict[u['class'][0]] = u.text
    
    
    return ref_dict

#this is the function for extracting the references from the tail
def extract_references(tail):
    
    references = []

    #find the references block
    refs = tail.find('div', {'tag':'ref-list'})

    #find the references list
    r = refs.find('ul')

    #go through the list of references
    for ref_el in r.children:

        #check if the item is whatever a navigable string is
        if 'bs4.element.NavigableString' in str(ref_el.__class__):
            #move on if it does
            continue

        ref_dict = process_reference_element(ref_el)
        
        if ref_dict != {}:
            references.append(ref_dict)
        
    return references

In [2]:
from bs4 import BeautifulSoup

#define a class to extract the acknowledgments
class acknowledgements_extractor(object):
    
    def __init__(self, tail):
        
        #get the acknowledgements element
        acks = tail.find('div', {'class':'ack'})

        #get the ack_dict element
        self.ack_dict = {'text':''}
        
        #start iterating
        self.iteracks(acks)
        
    def iteracks(self, el):
                
        #iterate through the children of this device
        for child in el.children:
            
            #check if the item is whatever a navigable string is
            if 'bs4.element.NavigableString' in str(child.__class__):
                #move on if it does
                continue
            
            #check if we have an div, usually the acknowledgments heading
            if child.name == 'div':

                #check if this element is a title
                if child['class'][0] == 'title':

                    self.ack_dict['title'] = child.text.strip()
                    
                    #if it is we are done with this element
                    continue
                    
                #go one level down
                self.iteracks(child)
            
            #check if this is a text element
            elif child.name == 'p':
                
                #now we need to extract and store the result
                self.ack_dict['text'] += ' ' + el.text.strip()


#this is the object that will process the paper and in which the results will be stored
class paper_obj(object):
    
    def __init__(self, html_file, keep_fig_caps = True):
        
        
        self.abstract_text = ''
        self.body_text = ''
        self.title = ''
        
        self.figures = []
        
        self.authors = []
        
        self.abstract_el = False
        self.body_el = False
        
        #tell the tool to keep the figure captions in the fulltexts
        self.keep_fig_labels = keep_fig_caps
        
        #load the data in
        with open(html_file, 'r') as f:

            raw_text = f.read()
            
            
        #parse the html into soup
        soup = BeautifulSoup(raw_text, 'html.parser')

        #extract the metadata
        front = soup.find('div', {'class':'front'})
        
        #extract the body
        body = soup.find('div', {'class':'body'})
        
        #extract the tail
        tail = soup.find('div', {'class':'back'})
        
        #extract the metadata into a json
        self.meta = self.iterhtml(front)
        
        #we are now going to work with the body so set the flag
        self.body_el = True
        
        #extract the body of the document
        self.body = self.iterhtml(body)
        
        #we are done with the body so we will reset the body to False
        self.body_el = False
        
        
        #extract the end notes
        self.extract_tail(tail)

    def iterhtml(self, el):
        
        #set up the json we're going to be returning
        store_json = {}
        
        #check if this item has children
        if el.children:

            for item in el.children:

                #check if the item is whatever a navigable string is
                if 'bs4.element.NavigableString' in str(item.__class__):
                    #move on if it does
                    continue

                if item.name == 'div':

                    #object_name = item.text

                    #check if we have a title object
                    if item.has_attr('tagx'):

                        if item['tagx'] == 'title':
                            store_json['title'] = item.text.strip()

                            #this is a heading, lets store the text from it
                            self.capture_text(item)
                            
                            #this is not a div element we are interested in diving into
                            #so lets move on
                            continue
                            
                    if item['class'][0] == 'fig':
                        
                        self.process_figure(item)
                        
                        #we dont want to dive in here since that will lead to double counts.
                        #lets move on
                        continue
                        
                    elif item['class'][0] == 'abstract':
                        
                        #this is going to be the abstract and we want to extract
                        #the text from it so lets use this function here to do that
                        self.process_abstract(item)
                        
                        #we are done here so lets move on since the diving in happened in
                        #the process abstract function
                        #continue
                    
                    #we have the title of the article
                    elif item['class'][0] == 'article-title':
                        
                        #store the title since the title is a unique class all of its own
                        store_json[item['class'][0]] = item.text.strip()
                        
                        self.title = item.text.strip()
                        
                        #this will not go any deeper so lets move on
                        continue
                        
                    #we have the authors information
                    elif item['class'][0] == 'contrib-group':
                        
                        #extract the authors from our author element
                        self.process_authors(item)
                        
                        #now move on since we're done with this element
                        continue
                        
                    #do some recursion
                    store_json[item['class'][0]] = self.iterhtml(item)
                    
                    #we dont want to store text from div elements so lets move on
                    #continue

                #we have a span object which contains information we are interested in
                elif item.name == 'span':
                    
                    #check if there is a link in this item
                    
                    links = item.findAll('a')
                    
                    if len(links) > 0:
                        
                        #
                        store_json[item['class'][0]] = {'text':item.text.strip(),
                                                       'links':[]}
                        
                        #go through this links we have in this span element
                        for link in links:
                            
                            store_json[item['class'][0]]['links'].append({'link':link['href'],
                                                                         'text':link.text.strip()})
                            
                    else:
                        #just store the text from the item
                        store_json[item['class'][0]] = item.text.strip()
                    
                    #grab the text from this element
                    self.capture_text(item)
                
                #if we have a list of the element
                elif item.name == 'p':
                    
                        
                    if 'text' not in store_json.keys():

                        store_json['text'] = ''

                    store_json['text'] += item.text
                    
                    #grab the text from this element
                    self.capture_text(item)
                 
        return store_json
    
    def capture_text(self, el):
        
        #check if we are working with a body
        if self.body_el:
            self.body_text += el.text + '\n'

        #if we have an abstract element we want to store it
        elif self.abstract_el:
            self.abstract_text += el.text + '\n'
            
    def process_figure(self, fig_el):
        
        fig_dict = {'caption':''}

        for child in fig_el.children:

            #check if the item is whatever a navigable string is
            if 'bs4.element.NavigableString' in str(child.__class__):
                #move on if it does
                continue

            if child.name == 'div':

                fig_dict['title'] = child.text
                
                if self.keep_fig_labels:
                    self.capture_text(child)

            elif child.name == 'p':
                #child.name == 'p':
                
                if self.keep_fig_labels:
                    self.capture_text(child)

                fig_dict['caption'] += child.text + '\n'
        
        #store the title and caption for this figure
        fig_dict['title'] = fig_dict['title'].strip()
        fig_dict['caption'] = fig_dict['caption'].strip()
        
        #store the details of this figure
        self.figures.append(fig_dict)


    def extract_tail(self, tail):
        
        ack_tool = acknowledgements_extractor(tail)
        
        self.acknowledgements = ack_tool.ack_dict
        
        self.references = extract_references(tail)
    
    
    def process_abstract(self, el):
        
        self.abstract_el = True
        
        self.iterhtml(el)
        
        self.abstract_el = False
    
    #process the list of authors
    def process_authors(self, el):
        
        #extract a list of author elements
        raw_authors = el.findAll('span', {'class':'contrib'})
    
        #move through the authors and extract them
        for author in raw_authors:
            
            #extract the details of this author
            author_dict = {}
            for stat in author.children:


                #if we have the person's link
                if stat.name == 'a':

                    author_dict['link'] = {'link':stat['href'],
                                          'text':stat.text.strip()}

                    continue

                #extract the name information if we are looking at the name element
                if stat['class'][0] == 'name':

                    for nam in stat.children:

                        author_dict[nam['class'][0]] = nam.text
                        
                #extract the other information if we have a non-name piece of info, e.g. email address
                else:

                    author_dict[stat['class'][0]] = stat.text.strip()

            #store what we've extracted about this author
            self.authors.append(author_dict)

In [6]:
#set the filename
filename = './results/PMC2241601/scholarly.html'

#create the paper object
result = paper_obj(filename)

print(result.abstract_text)


Background
The key enzymes of photosynthetic carbon assimilation in C4 plants have evolved independently several times from C3 isoforms that were present in the C3 ancestral species. The C4 isoform of phosphoenolpyruvate carboxylase (PEPC), the primary CO2-fixing enzyme of the C4 cycle, is specifically expressed at high levels in mesophyll cells of the leaves of C4 species. We are interested in understanding the molecular changes that are responsible for the evolution of this C4-characteristic PEPC expression pattern, and we are using the genus Flaveria (Asteraceae) as a model system. It is known that cis-regulatory sequences for mesophyll-specific expression of the ppcA1 gene of F. trinervia (C4) are located within a distal promoter region (DR).

Results
In this study we focus on the proximal region (PR) of the ppcA1 promoter of F. trinervia and present an analysis of its function in establishing a C4-specific expression pattern. We demonstrate that the PR harbours cis-regulatory det

In [4]:
filename = './results/PMC1920248/scholarly.html'

#create the paper object
paper = paper_obj(filename)

print(paper.abstract_text)

Calmodulin (CaM) is involved in defense responses in plants. In soybean (Glycine max), transcription of calmodulin isoform 4 (GmCaM4) is rapidly induced within 30 min after pathogen stimulation, but regulation of the GmCaM4 gene in response to pathogen is poorly understood. Here, we used the yeast one-hybrid system to isolate two cDNA clones encoding proteins that bind to a 30-nt A/T-rich sequence in the GmCaM4 promoter, a region that contains two repeats of a conserved homeodomain binding site, ATTA. The two proteins, GmZF-HD1 and GmZF-HD2, belong to the zinc finger homeodomain (ZF-HD) transcription factor family. Domain deletion analysis showed that a homeodomain motif can bind to the 30-nt GmCaM4 promoter sequence, whereas the two zinc finger domains cannot. Critically, the formation of super-shifted complexes by an anti-GmZF-HD1 antibody incubated with nuclear extracts from pathogen-treated cells suggests that the interaction between GmZF-HD1 and two homeodomain binding site repeat